In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from scipy import stats
from scipy.spatial.distance import pdist, cdist
from scipy.spatial import cKDTree
import seaborn as sns
import pandas as pd
import umap
from sklearn.manifold import MDS
from scipy.spatial.distance import squareform, pdist

from tqdm import tqdm
import os

from src.models import SpaceNet, RecurrentSpaceNet, Decoder
from src.utils import ratemap_collage, SimpleDatasetMaker

plt.rcdefaults()
plt.style.use("figures/project_style.mplstyle")
%matplotlib inline

In [2]:
figure_path = os.path.join(os.getcwd(), "figures")
model_path = os.path.join(os.getcwd(), "models")
results_path = os.path.join(os.getcwd(), "results")

# Recurrent Network (no context)

In [3]:
# ----------------------- Params -----------------------
train_steps = 60000         # Number of training steps
timesteps = 10              # Length of trajectories
bs = 64                     # Batch size
lr = 1e-4                   # Learning rate
n_models = 1                # Number of models to train

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = {
    # Default model
    "256": [RecurrentSpaceNet(n_in=2, n_out=256, corr_across_space=True, scale=0.25, beta=0.5, device=device) for _ in range(n_models)],
    
    # Beta grid
    "256_0beta": [RecurrentSpaceNet(n_in=2, n_out=256, corr_across_space=True, scale=0.25, beta=0., device=device) for _ in range(n_models)],
    "256_025beta": [RecurrentSpaceNet(n_in=2, n_out=256, corr_across_space=True, scale=0.25, beta=0.25, device=device) for _ in range(n_models)],
    "256_075beta": [RecurrentSpaceNet(n_in=2, n_out=256, corr_across_space=True, scale=0.25, beta=0.75, device=device) for _ in range(n_models)],
    
    # Scale grid
    "256_01scale": [RecurrentSpaceNet(n_in=2, n_out=256, corr_across_space=True, scale=0.1, beta=0.5, device=device) for _ in range(n_models)],
    "256_05scale": [RecurrentSpaceNet(n_in=2, n_out=256, corr_across_space=True, scale=0.5, beta=0.5, device=device) for _ in range(n_models)],
    
    # n grid
    "512": [RecurrentSpaceNet(n_in=2, n_out=512, corr_across_space=True, scale=0.25, beta=0.5, device=device) for _ in range(n_models)],
    "1024": [RecurrentSpaceNet(n_in=2, n_out=1024, corr_across_space=True, scale=0.25, beta=0.5, device=device) for _ in range(n_models)],
}

loss_histories = {name: [] for name in models.keys()}

# --------------------- Training ----------------------

for name, model_list in models.items():
    
    print(f"Training {name}")
    for i, model in enumerate(model_list):
        
        print(f"Model {i+1}")
        
        # Initialize optimizer and dataset generator
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        genny = SimpleDatasetMaker()    
        
        if os.path.exists(os.path.join(model_path, f"{name}_{i}.pt")):
            model = torch.load(os.path.join(model_path, f"{name}_{i}.pt"))
            loss_history = np.load(os.path.join(model_path, f"{name}_{i}_loss_history.npy"))
            loss_histories[name].append(loss_history)
            continue
        
        loss_history = []
        progress = tqdm(range(train_steps))
        for k in progress:  
            
            # Create batch of trajectories
            r, v = genny.generate_dataset(bs, timesteps, device=device)
        
            # Perform training step
            loss = model.train_step(x=(v, r[:, 0]), y=r[:, 1:], optimizer=optimizer)
        
            loss_history.append(loss)
            
            if k % 10 == 0:
                progress.set_description(f"loss: {loss:>7f}")
                
        models[name][i] = model
        loss_histories[name].append(loss_history)

        # Save model and loss history
        torch.save(model, os.path.join(model_path, f"{name}_{i}.pt"))
        np.save(os.path.join(model_path, f"{name}_{i}_loss_history.npy"), loss_history)


Training 256
Model 1
Training 256_0beta
Model 1
Training 256_025beta
Model 1
Training 256_075beta
Model 1
Training 256_01scale
Model 1
Training 256_05scale
Model 1
Training 512
Model 1
Training 1024
Model 1


# Recurrent Network (context)

In [9]:
# ----------------------- Params -----------------------
train_steps = 60000             # Number of training steps
timesteps = 10                  # Length of trajectories
bs = 64                         # Batch size
lr = 1e-4                       # Learning rate
n_models = 1                    # Number of models to train
cmin = -2                       # Minimum context value
cmax = 2                        # Maximum context value

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = {
    "256_context": [RecurrentSpaceNet(n_in=3, n_out=256, corr_across_space=True, scale=0.25, beta=0.5, device=device, initial_state_size=3) for _ in range(n_models)],
    "256_context_not_initial": [RecurrentSpaceNet(n_in=3, n_out=256, corr_across_space=True, scale=0.25, beta=0.5, device=device, initial_state_size=2) for _ in range(n_models)],
    
    # Beta grid
    "256_context_0beta": [RecurrentSpaceNet(n_in=3, n_out=256, corr_across_space=True, scale=0.25, beta=0., device=device, initial_state_size=3) for _ in range(n_models)],
    "256_context_025beta": [RecurrentSpaceNet(n_in=3, n_out=256, corr_across_space=True, scale=0.25, beta=0.25, device=device, initial_state_size=3) for _ in range(n_models)],
    "256_context_075beta": [RecurrentSpaceNet(n_in=3, n_out=256, corr_across_space=True, scale=0.25, beta=0.75, device=device, initial_state_size=3) for _ in range(n_models)],
}

loss_histories = {name: [] for name in models.keys()}

# --------------------- Training ----------------------

for name, model_list in models.items():
    
    print(f"Training {name}")
    for i, model in enumerate(model_list):
        
        print(f"Model {i+1}")
        
        context_in_initial = True if model.initial_state_size == 3 else False
        
        # Initialize optimizer and dataset generator
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        genny = SimpleDatasetMaker()    
        
        if os.path.exists(os.path.join(model_path, f"{name}_{i}.pt")):
            model = torch.load(os.path.join(model_path, f"{name}_{i}.pt"))
            loss_history = np.load(os.path.join(model_path, f"{name}_{i}_loss_history.npy"))
            loss_histories[name].append(loss_history)
            continue
        
        loss_history = []
        progress = tqdm(range(train_steps))
        for k in progress:  
            
            # Create batch of trajectories
            r, v = genny.generate_dataset(bs, timesteps, device=device)
            
            # Get random contexts and use for all timesteps along a trajectory
            c = torch.tensor(np.random.uniform(cmin, cmax, bs), dtype=torch.float32, device=device)
            c = c[:, None, None] * torch.ones((1, timesteps - 1, 1), device=device)
            
            # Build initial input
            if context_in_initial:
                initial_input = torch.cat((r[:, 0], c[:, 0]), dim=-1)
            else:
                initial_input = r[:, 0]
            
            # Concatenate velocity and context
            inputs = (torch.cat((v, c), dim=-1), initial_input)
            labels = (r[:, 1:], c)
        
            # Perform training step
            loss = model.train_step(x=inputs, y=labels, optimizer=optimizer)
        
            loss_history.append(loss)
            
            if k % 10 == 0:
                progress.set_description(f"loss: {loss:>7f}")
                
        models[name][i] = model
        loss_histories[name].append(loss_history)

        # Save model and loss history
        torch.save(model, os.path.join(model_path, f"{name}_{i}.pt"))
        np.save(os.path.join(model_path, f"{name}_{i}_loss_history.npy"), loss_history)


Training 256_context
Model 1


loss: 0.000619: 100%|██████████| 60000/60000 [16:38<00:00, 60.09it/s]


Training 256_context_not_initial
Model 1


loss: 0.002729:   6%|▋         | 3797/60000 [01:33<22:59, 40.74it/s]


KeyboardInterrupt: 

# Feedforward Network

In [6]:
# ----------------------- Params -----------------------
train_steps = 60000             # Number of training steps
bs = 64                         # Batch size
lr = 1e-4                       # Learning rate
n_models = 1                    # Number of models to train

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = {
    "256_ff": [SpaceNet(n_in=2, n_out=256, scale=0.25, beta=0.5, device=device) for _ in range(n_models)],
}

loss_histories = {name: [] for name in models.keys()}

# --------------------- Training ----------------------

for name, model_list in models.items():
    
    print(f"Training {name}")
    for i, model in enumerate(model_list):
        
        print(f"Model {i+1}")
        
        # Initialize optimizer and dataset generator
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        genny = SimpleDatasetMaker()    
        
        if os.path.exists(os.path.join(model_path, f"{name}_{i}.pt")):
            model = torch.load(os.path.join(model_path, f"{name}_{i}.pt"))
            loss_history = np.load(os.path.join(model_path, f"{name}_{i}_loss_history.npy"))
            loss_histories[name].append(loss_history)
            continue
        
        loss_history = []
        progress = tqdm(range(train_steps))
        for k in progress:  
            
            # Create batch of positions
            r = torch.tensor(np.random.uniform(-1, 1, (bs, 2)), dtype=torch.float32)
        
            # Perform training step
            loss = model.train_step(x=r, y=r, optimizer=optimizer)
        
            loss_history.append(loss)
            
            if k % 10 == 0:
                progress.set_description(f"loss: {loss:>7f}")
                
        models[name][i] = model
        loss_histories[name].append(loss_history)

        # Save model and loss history
        torch.save(model, os.path.join(model_path, f"{name}_{i}.pt"))
        np.save(os.path.join(model_path, f"{name}_{i}_loss_history.npy"), loss_history)


Training 256_ff
Model 1


loss: 0.000147: 100%|██████████| 60000/60000 [00:49<00:00, 1203.60it/s]


## Feedforward Network (context)

In [3]:
# ----------------------- Params -----------------------
train_steps = 60000             # Number of training steps
bs = 64                         # Batch size
lr = 1e-4                       # Learning rate
n_models = 1                    # Number of models to train
cmin = -2                       # Minimum context value
cmax = 2                        # Maximum context value

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = {
    "256_ff_context": [SpaceNet(n_in=3, n_out=256, scale=0.25, beta=0.5, device=device) for _ in range(n_models)],
}

loss_histories = {name: [] for name in models.keys()}

# --------------------- Training ----------------------

for name, model_list in models.items():
    
    print(f"Training {name}")
    for i, model in enumerate(model_list):
        
        print(f"Model {i+1}")
        
        # Initialize optimizer and dataset generator
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        genny = SimpleDatasetMaker()    
        
        if os.path.exists(os.path.join(model_path, f"{name}_{i}.pt")):
            model = torch.load(os.path.join(model_path, f"{name}_{i}.pt"))
            loss_history = np.load(os.path.join(model_path, f"{name}_{i}_loss_history.npy"))
            loss_histories[name].append(loss_history)
            continue
        
        loss_history = []
        progress = tqdm(range(train_steps))
        for k in progress:  
            
            # Create batch of positions and contexts            
            r = torch.tensor(np.random.uniform(-1, 1, (bs, 2)), dtype=torch.float32)
            c = torch.tensor(np.random.uniform(cmin, cmax, bs), dtype=torch.float32)[:, None]
            inputs = torch.cat((r, c), dim=-1)
            labels = (r, c)
                
            # Perform training step
            loss = model.train_step(x=inputs, y=labels, optimizer=optimizer)
        
            loss_history.append(loss)
            
            if k % 10 == 0:
                progress.set_description(f"loss: {loss:>7f}")
                
        models[name][i] = model
        loss_histories[name].append(loss_history)

        # Save model and loss history
        torch.save(model, os.path.join(model_path, f"{name}_{i}.pt"))
        np.save(os.path.join(model_path, f"{name}_{i}_loss_history.npy"), loss_history)


Training 256_ff_context
Model 1


loss: 0.000514: 100%|██████████| 60000/60000 [01:11<00:00, 834.45it/s] 
